In [19]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
import pandas as pd

In [20]:
def LatLonAlt2XYZ(Lat : float,Lon : float,Alt : float) -> tuple:
    a_e = 6378137
    ecc = 0.081819190842622
    ksi = 1.0 / np.sqrt(1 - ecc * ecc * np.sin(Lat) * np.sin(Lat))
    X = (a_e * ksi + Alt) * np.cos(Lat) * np.cos(Lon)
    Y = (a_e * ksi + Alt) * np.cos(Lat) * np.sin(Lon)
    Z = (a_e * ksi * (1 - ecc * ecc) + Alt) * np.sin(Lat)
    return X, Y, Z

In [21]:
def CalcGravity(X:float,Y:float,Z:float)->np.array:
    t2 = X ** 2.0
    t3 = Y ** 2.0
    t4 = Z ** 2.0
    t5 = t2 + t3 + t4
    t6 = 1.0 / t5 ** (3.0 / 2.0)
    t7 = 1.0 / t5 ** (5.0 / 2.0)
    t8 = 1.0 / t5 ** (7.0 / 2.0)
    g = np.array([
       X * 5.3174941173225e-9-X * t6 * 3.9860142624e14-X * t7 * 2.633274580483414e25 + X * t4 * t8 * 1.316637290241707e26,
       Y * 5.3174941173225e-9 - Y * t6 * 3.9860142624e14 - Y * t7 * 2.633274580483414e25 + Y * t4 * t8 * 1.316637290241707e26,
       Z * t6 * -3.9860142624e14 - Z * t7 * 7.899823741450242e25 + Z * t4 * t8 * 1.316637290241707e26])
    return g

In [22]:
def SkewSymmMatr(a : np.array) -> np.array:
    Ax= np.array(
        [[0.0,-a[2],a[1]],
        [a[2],0.0,-a[0]],
        [-a[1],a[0],0.0]])
    return Ax

In [23]:
def cross(a : np.array ,b : np.array) -> np.array:
    axb = SkewSymmMatr(a) @ b
    return axb

In [24]:
def twoPiBound(heading : float) -> float:
    if heading < 0:
        heading = heading + 2.0 * np.pi

    return heading

In [25]:
def DCM_bn(heading : float, pitch : float, roll : float) -> np.array:
    sz = np.sin(pitch)
    cz = np.cos(pitch)
    sy = np.sin(-heading)
    cy = np.cos(-heading)
    sx = np.sin(roll)
    cx = np.cos(roll)

    Cx = np.array([[1, 0, 0],[0, cx, sx],[0, -sx, cx]],dtype = np.float64)
    Cy = np.array([[cy, 0, -sy],[0, 1, 0],[sy, 0, cy]],dtype = np.float64)
    Cz = np.array([[cz, sz, 0],[-sz, cz, 0],[0, 0, 1]],dtype = np.float64)

    Cnb = Cx @ Cz @ Cy

    return Cnb.T

In [26]:
def Calc_F_C_N(B : float,h : float,a_e : float,ecc2 : float) -> np.array:
    sinB = np.sin(B)
    sinB2 = sinB * sinB

    R_N = a_e * (1 - ecc2) / (1 - ecc2 * sinB2) ** 1.5 + h

    R_E = a_e / np.sqrt(1 - ecc2 * sinB2) + h

    F_C_N = np.array([
        [0.0,0.0,1.0 / R_E],
        [0.0,0.0,np.tan(B)/R_E],
        [-1.0/R_N,0.0,0.0]])

    return F_C_N

In [27]:
def CalcDCMnue_align(Pos : list,fb : np.array,om_ib : np.array) -> np.array:
    #nue->ned
    fb =np.array([fb[0],fb[2],-fb[1]])
    om_ib = np.array([om_ib[0],om_ib[2],-om_ib[1]])

    om_ie = 7.292115e-5

    cosB = np.cos(Pos[0])
    sinB = np.sin(Pos[0])
    cosL = np.cos(Pos[1])
    sinL = np.sin(Pos[1])

    Lat = Pos[0]

    (X,Y,Z) = LatLonAlt2XYZ(Pos[0],Pos[1],Pos[2])
    gravity = CalcGravity(X,Y,Z)

    gt = -cosL*sinB*gravity[0] - sinL*sinB*gravity[1] + cosB*gravity[2]
    gn = -(cosL*cosB*gravity[0] + sinL*cosB*gravity[1] + sinB*gravity[2])

    b1 = -fb
    b2 = om_ib
    b3 = cross(b1, b2)

    Mb = np.array([b1,b2,b3])

    Mn = np.array([[np.sin(Lat) / (gn * np.cos(Lat) + gt * np.sin(Lat)),gn / (gn * om_ie * np.cos(Lat) + gt * om_ie * np.sin(Lat)),0 ],
                  [0,0, 1 / (gn * om_ie * np.cos(Lat) + gt * om_ie * np.sin(Lat))],
                  [np.cos(Lat) / (gn * np.cos(Lat)+ gt * np.sin(Lat)),  -gt / (gn * om_ie * np.cos(Lat) + gt * om_ie * np.sin(Lat)), 0 ]] )

    R_nb = Mn @ Mb

    DCM = R_nb.T

    return DCM

In [28]:
def threeaxisrot(r11, r12, r21, r31, r32):
    r1 = np.arctan2(r11,r12)
    r2 = np.arcsin(r21)
    r3 = np.arctan2(r31,r32)
    return r1,r2,r3

def EulerFromDCMnue(dcm):
    (r1, r2, r3) = threeaxisrot(dcm[0, 1],dcm[0,0], -dcm[0, 2],dcm[1, 2], dcm[2, 2])
    return (r1, r2, r3)

### Загрузка данных

In [29]:
mat = loadmat('data/nav.binB_03.mat')
data_df = pd.DataFrame(mat['UAV'])
data = data_df.to_numpy().T
data = np.delete(data, 0, 1)
data = data[405500:,:]
Sensors = np.zeros((6,2),dtype = np.float64)
n = data.shape[0]
acc = data[:,9:12]
gyro = data[:,12:15]

GPSQuality = data[:,23]
GPS = data[:,17:23]
GPS[:,0:2] = GPS[:,0:2] * np.pi / 180

StatusMS = data[:, 33]
MagnSens = data[:, 30:33] * 10
MagnSens[:,1:3] = -1 * MagnSens[:,1:3]

dt = 1 / 100
deg_to_rad = np.pi/180
rad_to_deg = 1/deg_to_rad

## Выставка

In [30]:
Alignment = 60000
Lat = GPS[Alignment,0]
Lon = GPS[Alignment,1]
Alt = GPS[Alignment,2]
W_NUE = GPS[Alignment,3:6]
W_NUE_old = W_NUE
Lat_old = Lat
Alt_old = Alt
Roll = 0
Pitch = 0
Heading = 0
C_B_N = np.eye(3)
NavState = np.zeros(26)
MS_mean_count = 0
gyros_mean_count = 0
accels_mean = np.zeros(3)
gyros_mean = np.zeros(3)
MS_mean = np.zeros(3)

### Накопление данных

In [31]:
for i in range(Alignment):
    accels_mean = accels_mean + acc[i, :]
    gyros_mean = gyros_mean + gyro[i, :]
    gyros_mean_count = gyros_mean_count + 1
    if (StatusMS[i] == 2):
        MS_mean = MS_mean + MagnSens[i]
        MS_mean_count = MS_mean_count + 1

### Расчет углов крена и тангажа,определение курса методом гирокомпасирования

In [32]:
accels_mean = (accels_mean/gyros_mean_count) / dt # Осреднение показаний акселерометров. Используется счетчик данных гироскопа
gyros_mean = (gyros_mean/gyros_mean_count) / dt # Осреднение показаний гироскопов + пересчет в рад/сек
MS_mean = (MS_mean / MS_mean_count)
# если датчики - ВОГ, то проводится определение курса методом гирокомпасирования
Rbn = CalcDCMnue_align([Lat,Lon,Alt],accels_mean,gyros_mean)
(Heading, Pitch, Roll) = EulerFromDCMnue(Rbn)
C_B_N = DCM_bn(Heading,Pitch,Roll) # Расчет DCM
Heading = twoPiBound(Heading)  # Ограничение
C_I_B = C_B_N.T
C_I_N = C_B_N.T
Sensors = np.zeros((6,),dtype = np.float64)

In [33]:
Sensors

array([0., 0., 0., 0., 0., 0.])

In [34]:
print("Параметры после выставки")
print("Курс ")
print(Heading * (180 / np.pi))
print("Тангаж ")
print(Pitch * (180 / np.pi))
print("Крен ")
print(Roll * (180 / np.pi))
print("Широта ")
print(Lat * (180 / np.pi))
print("Долгота ")
print(Lon * (180 / np.pi))
print("Высота ")
print(Alt)
print("W_N ")
print(W_NUE[0])
print("W_U ")
print(W_NUE[1])
print("W_E ")
print(W_NUE[2])

Параметры после выставки
Курс 
333.2617518321868
Тангаж 
6.023054025873441
Крен 
-0.8737296834068249
Широта 
54.9797978
Долгота 
37.663690100000004
Высота 
192.796
W_N 
-0.054999999701976776
W_U 
-0.0430000014603138
W_E 
0.08799999952316284


In [37]:
NavState = np.array([Lat,Lon,Alt,          # Широта, долгота, геодезическая высота
                   W_NUE[0],W_NUE[1],W_NUE[2]])           # Скорость на север, вверх, на восток

In [38]:
# Статические переменные
Om_e = 7.292115e-5
E = np.array([[1.0,0.0,0.0],[0.0,1.0,0.0],[0.0,0.0,1.0]],dtype = np.float64)
# Используется ПЗ-90
a_e = 6378136.0
ecc = 0.0818191065283638
ecc2 = ecc * ecc
iter = 0
delta_v_array = np.zeros((3,4))
delta_alfa_array = np.zeros((3,4))

In [39]:
delta_v_array

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [40]:
def SINS(NavState : np.array, Sensors : np.array, dt : int, C_i_b : np.array, C_i_n : np.array ) -> tuple:

    global Om_e
    global E

    # Используется ПЗ-90
    global a_e
    global ecc2
    global iter
    global delta_v_array
    global delta_alfa_array

    # переменные квази-скоростей и квази-угллов
    delta_v = Sensors[0:3,0] # выходной сигнал акселерометра по осям чувствительности (x,y,z) на текущем такте расчетов
    delta_alpha = Sensors[3:6,0] # выходной сигнал гироскопов (скорость) по оси (x,y,z) на текущем такте расчетов
    delta_v_array[0:3,i] = delta_v
    delta_alfa_array[0:3,i] = delta_alpha
    iter += 1

    if iter == 3:

        B = NavState[0]
        L = NavState[1]  # текущая  долгота ВС в выбранной СК
        h = NavState[2]  # текущая  геодезическая высота ВС относительно эллипсоида выбранной СК
        W = NavState[3:6] # текущий вектор земной скорости ВС (N,U,E)
        sinB = np.sin(B)
        cosB = np.cos(B)
        sinL = np.sin(L)
        cosL = np.cos(L)
        sinB2 = sinB * sinB
        R_N = a_e * (1 - ecc2) / (1 - ecc2 * sinB2) ** 1.5 + h
        R_E = a_e / np.sqrt(1 - ecc2 * sinB2) + h
        # Расчет вектора конечного поворота Эйлера (body)
        alfa_1_2 = np.array([delta_alpha_array[0,0],delta_alpha_array[1,0], delta_alpha_array[2,0]])+np.array([delta_alpha_array[0,1],delta_alpha_array[1,1], delta_alpha_array[2,1]])
        alfa_3_4 = np.array([delta_alpha_array[0,2],delta_alpha_array[1,2], delta_alpha_array[2,2]])+np.array([delta_alpha_array[0,3],delta_alpha_array[1,3], delta_alpha_array[2,3]])
        alfa = alfa_1_2 + alfa_3_4
        beta = 2 / 3 * cross(alfa_1_2,alfa_3_4) # коннинг
        alfa = alfa + beta # вектор порота body эйлера
        skew_alfa = SkewSymmMatr(alfa) # Коссосиметричная матрица
        mod_alpha = np.sqrt(delta_alpha[0] * delta_alpha[0] + delta_alpha[1] * delta_alpha[1] + delta_alpha[2] * delta_alpha[2])
        mod_alpha2 = mod_alpha * mod_alpha
        # Расчет вектора конечного поворота (скорости) Эйлера (Navigation)
        omega_IE_N = np.array([Om_e * cosB, Om_e * sinB, 0.0 ])
        F_C_N = Calc_F_C_N(B, h, a_e, ecc2)
        omega_sum = omega_IE_N + F_C_N @ W
        skew_omega = SkewSymmMatr(omega_sum)
        # нахождение МНК из body t-1 в body t при помощи решения уравенения Пуасона
        C_b_b = E + (1 - mod_alpha2/6)*skew_alfa + (0.5 - mod_alpha2/24) * skew_alfa @ skew_alfa
        C_i_b = C_b_b @ C_i_b  # Получили МНК из body в инерциальную в данный такт
        # нахождение МНК из навигационной СК t-1 в навигационную СК t при помощи решения уравенения Пуасона
        C_n_n = E - dt * skew_omega + (dt**2) / 2 * skew_omega @ skew_omega
        C_i_n = C_n_n @ C_i_n
        C_b_n = C_i_n @ C_i_b.T
        #нормализация и ортогонализация мнк
        C_i_b = (E - ((1.0 / 2.0) * ( C_i_b @  C_i_b.T - E))) @  C_i_b
        C_i_n = (E - ((1.0 / 2.0) * ( C_i_n @  C_i_n.T - E))) @  C_i_n
        C_b_n = (E - ((1.0 / 2.0) * ( C_b_n @  C_b_n.T - E))) @  C_b_n
        #нахождение приращения линейных скоростей методом Рунге-Кутта 4-го порядка
        delta_W = np.array([0,0,0])
        for i in range(4):
            skew_delta_alfa = SkewSymmMatr(delta_alfa_array[0:2,i])
            w_b = delta_v_array[0:2,i]
            k_1 = w_b - delta_alpha_sym @ delta_W
            k_2 = w_b - delta_alpha_sym @ (delta_W + (dt / 2) * k_1)
            k_3 = w_b - delta_alpha_sym @ (delta_W + (dt / 2) * k_2)
            k_4 = w_b - delta_alpha_sym @ (delta_W + dt * k_3)
            delta_W = delta_W + (k_1+2*k_2+2*k_3+k_4)/6
        delta_W_n = C_b_N @ delta_W

    return NavState ,gt ,C_i_b , C_i_n